# Esercizio 3 - Sequence tree

utility


In [1]:
from datetime import timedelta
from datetime import datetime

# Some helper functions!

def format_timedelta(td: timedelta) -> str:
    days = td.days
    years, days = divmod(days, 365)
    months, days = divmod(days, 30)
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    formatted_str = ""
    if years:
        formatted_str += f"{years}y "
    if months:
        formatted_str += f"{months}mo "
    if days:
        formatted_str += f"{days}d "
    if hours:
        formatted_str += f"{hours}h "
    if minutes:
        formatted_str += f"{minutes}m "
    if seconds:
        formatted_str += f"{seconds}s "
    

    
    return formatted_str[:-1] if formatted_str else "0s"

    
def find_durations(dataset):
    lengths = []
    for entry in dataset:
        min_t = datetime.max
        max_t = datetime.min
        for item in entry:
            if item[0] < min_t:
                min_t = item[0]
            elif item[0] > max_t:
                max_t = item[0]
        lengths.append(max_t-min_t)
    return [(i, x) for i, x in enumerate(lengths)]

## Operazioni preliminari sul DS

#### Carichiamo il DS

In [2]:
import os
from datetime import datetime

def load_diabetes_dataset(verbose=False)-> list: 
    folder_path="datasets\\diabetes"
    dataset = []
    errcount=0
    print(f"-- DS loader")

    for filename in os.listdir(folder_path):

        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path)and filename.startswith('data'):
            entry=[]

            with open(file_path, 'r') as file:
                content = file.readlines()
                for line in content:
                    item = tuple((line[0:-1] if line.endswith('\n') else tuple(line)).split("\t"))

                    # If the item is valid, append it to the entry
                    try:
                        item_f = datetime.strptime(item[0]+" "+item[1], "%m-%d-%Y %H:%M")
                        entry.append((item_f,item[2],item[3]))
                    except:
                        if(verbose):
                            print(f"\t[!] Entry {item} in file {filename} is NOT vallid. Skipped!")
                        errcount+=1
                # add the entry to the dataset
                dataset.append(entry)
    print(f"\tSkipped {errcount} items for formatting issues in data file. {len(dataset)} loaded.")
    return dataset

dataset = load_diabetes_dataset(False)


-- DS loader
	Skipped 46 items for formatting issues in data file. 70 loaded.


#### A partire dal dataset grezzo generiamo X,Y,ST e V.

> Definiamo la classe reale come uno [0,1] che indica se l'evento 65 (65 = Hypoglycemic symptoms) si è verificato in una certa finestra di tempo `event_window` dopo un certo waiting time `waiting_window`.

In [6]:
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta

def compute_datasets(dataset:list,observation_window,waiting_window,prediction_window)->list:

    dataset_X = []
    dataset_Y = []
    dataset_F = []

    dataset_ST = [entry[0][0] for entry in dataset ]

    count_excluded=0

    for i in range(0,len(dataset)):
        entry = dataset[i]

        end_obs = dataset_ST[i]+observation_window
        
        start_pred = end_obs + waiting_window
        end_pred = start_pred + prediction_window

        if end_pred < entry[-1][0]:
            entry_X = []
            found = 0

            for item in entry:
                if item[0]>= dataset_ST[i] and item[0]<end_obs:
                    entry_X.append(item)
                if item[0]>=start_pred and item[0]<end_pred:
                    # put Y=1 if it has at least one "65" entry
                    if (item[1]=="65"):
                        found = 1
            dataset_X.append(entry_X)
            dataset_Y.append(found)

        else:
            count_excluded+=1
    
    dataset_ST = [entry[0][0] for entry in dataset_X ]
    dataset_V = [None]*len(dataset_X)
    print(f"-- DS builder")
    print(f"\t{count_excluded} entries unsuitable for selected windows.")
    print(f"\tFinal dataset size: {len(dataset_X)}. Classes: {sum(1 for c in dataset_Y if c == 1)}|{sum(1 for c in dataset_Y if c == 0)}, entropy {compute_entropy(dataset_Y):4.3}")    
    return dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F

dataset = load_diabetes_dataset()
observation_window = timedelta(days=+5)
waiting_window = timedelta(days=+5)
prediction_window = timedelta(days=+15)

# Convert raw data into dataset X,Y, etc
dataset_X,dataset_Y, dataset_ST,dataset_V, dataset_F = compute_datasets(dataset[:],observation_window,waiting_window,prediction_window)



-- DS loader
	Skipped 46 items for formatting issues in data file. 70 loaded.
-- DS builder
	6 entries unsuitable for selected windows.
	Final dataset size: 64. Classes: 27|37, entropy 0.982


#### Stampiamo il DS elaborato.

In [7]:
def print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,indexes=None):
    print("--DS state")
    if (indexes and len(indexes)>len(dataset_X)):
        indexes=None
    
    print("index\tX\tY\tST\t\t\tV")

    if indexes is None:
        indexes = range(0,len(dataset_X))

    for i in indexes:
        print(f"{i}\tl.{len(dataset_X[i])}\t{dataset_Y[i]}\t{dataset_ST[i]}\t{dataset_V[i]}")
    print(f"# entries: {len(dataset_X)}, entropy={compute_entropy(dataset_Y):4.3}")

    
print_dataset_state(dataset_X,dataset_Y, dataset_ST,dataset_V)

--DS state
index	X	Y	ST			V
0	l.35	1	1991-04-21 09:09:00	None
1	l.38	0	1989-10-10 08:00:00	None
2	l.49	1	1990-07-21 06:43:00	None
3	l.41	0	1990-08-19 17:00:00	None
4	l.46	0	1990-09-01 16:48:00	None
5	l.38	0	1989-03-27 22:00:00	None
6	l.24	1	1990-07-31 12:09:00	None
7	l.43	0	1990-04-22 18:08:00	None
8	l.34	0	1989-02-18 08:00:00	None
9	l.34	1	1990-07-13 09:44:00	None
10	l.46	1	1990-07-22 09:53:00	None
11	l.58	1	1990-09-04 05:53:00	None
12	l.38	1	1991-03-11 18:15:00	None
13	l.27	1	1991-04-13 08:47:00	None
14	l.22	1	1991-05-22 07:24:00	None
15	l.40	1	1990-07-13 09:48:00	None
16	l.44	0	1990-08-18 07:16:00	None
17	l.45	1	1990-09-09 17:23:00	None
18	l.39	0	1991-05-12 06:55:00	None
19	l.44	0	1989-09-03 08:00:00	None
20	l.36	0	1991-03-14 22:05:00	None
21	l.37	0	1991-04-27 23:02:00	None
22	l.28	0	1991-05-28 21:35:00	None
23	l.13	0	1990-07-24 16:00:00	None
24	l.25	0	1988-07-13 08:00:00	None
25	l.16	0	1989-01-29 08:00:00	None
26	l.16	0	1989-11-05 07:00:00	None
27	l.40	0	1990-04-29 07:00:00	None
28

In [31]:
import itertools
import random
import math

random.seed(1)

# Find al possible d,l couples that I could split the tree on. Note: Ds are randomly selected bc otherwise I'd end up with ~36000 pairs...
def create_pairs(dataset_X:list,dataset_ST:list,howmany_d =30,random_sampling=False):

    #1. Create all labels available from current
    labels = set()
    for i in range(0,len(dataset_X)):
        for item in dataset_X[i]:
            if (item[0] > dataset_ST[i]):# Only consider label if it's not been superato
                labels.add(item[1])


    #2. Find all d
    durations = set()
    if random_sampling:
        durations = set()
        for i in range(0,len(dataset_X)):
            for item in dataset_X[i]:
                if (item[0] > dataset_ST[i]): # Only consider timestamp if it's not been superato
                    durations.add(item[0]-dataset_ST[i])
        print(len("aaaaaaaaaaaa "),durations)
        if len(durations)>howmany_d:
            durations = random.sample(sorted(durations),howmany_d) # Is this ok?
        else:
            durations = sorted(durations)
    else:
        durations = set()
        for i in range(0,len(dataset_X)):
            for item in dataset_X[i]:
                durations.add(item[0]-dataset_ST[i])
        delta = max(durations)/(howmany_d+1)
        durations = set()
        for i in range(1,howmany_d+1):
            durations.add(delta*i)

    #print([format_timedelta(x) for x in sorted(durations)])
    return sorted(list(itertools.product(durations,labels)))

# Given a set with binary classes, computes entropy
def compute_entropy(dataset_Y):
    ones = len(list(filter(lambda classification : classification == 1,dataset_Y)))
    zeros = len(list(filter(lambda classification : classification == 0,dataset_Y)))

    if(ones == 0 or zeros==0):
        return 0
    
    entropy = ones/len(dataset_Y)*math.log2(1/(ones/len(dataset_Y))) + zeros/len(dataset_Y)*math.log2(1/(zeros/len(dataset_Y)))

    return entropy

def test_event(dataset_X,dl_pair,dataset_ST,dataset_V=None,update=False):
    i_T = [] # indexes of entries that have label==l within d time
    i_F = [] # indexes of entries that have DON'T HAVE label==l within d time
    d,l = dl_pair


    #1. Separate entries that satisfy event test from those who don't
    for i in range(0,len(dataset_X)):
        entry = dataset_X[i]
        found=False

        for item in entry:
            #print(item)
            if(found is False and item[0]>=dataset_ST[i] and item[0]<=(dataset_ST[i]+d) and item[1]==l ):
                found=True

        if(found):
            i_T.append(i)
        else:
            i_F.append(i)
    return i_T,i_F

# Given a pair of duration d and label l, computes its information gain on the dataset if we were to split it according to the sequence tree rules.
def compute_IG(dl_pair,dataset_X,dataset_Y,dataset_ST,verbose=False):
    #if verbose:
     #   print(f"Computing IG for {dl_pair}")

    entropy_0 = compute_entropy(dataset_Y)

    i_T, i_F = test_event(dataset_X,dl_pair,dataset_ST)
    
    #2. Compute final entropy. first let's generate our new datasets...
    dataset_Yt=[ dataset_Y[i] for i in i_T]
    dataset_Yf=[ dataset_Y[i] for i in i_F]
            
    entropy_f = (len(i_T)/len(dataset_X))*compute_entropy(dataset_Yt) + (len(i_F)/len(dataset_X))*compute_entropy(dataset_Yf)

    if verbose:
        print(f"information gain is {entropy_0-entropy_f} {entropy_0}-> {[ dataset_Y[i] for i in i_T],[ dataset_Y[i] for i in i_F]} {entropy_f}")
        
    return(entropy_0-entropy_f,i_T,i_F)


# Given all possible pairs of d,l finds the one with the highest information gain (aka the one I should actually split on)
def maximize_IG(dataset_X,dataset_Y, dataset_ST, indexes=None,verbose=False,howmany=30,random_sampling=False):
    if indexes is None:
        indexes = range(0,len(dataset_X))

    
    dl_pairs = create_pairs([dataset_X[i] for i in indexes],[dataset_ST[i] for i in indexes],howmany,random_sampling)

    #igs_dict = {x:compute_IG(x,dataset_X,dataset_Y,dataset_ST,verbose) for x in dl_pairs}
    igs_list = [(x,compute_IG(x,dataset_X,dataset_Y,dataset_ST,verbose)) for x in dl_pairs]
    # ogni entry di igs_list è ( (d,l) , (ig,i_T,i_F)   )

    max_ig=-1
    max_dl=None

    for ((d,l),(ig,i_T,i_F)) in igs_list:
        if ig > max_ig:
            max_ig=ig
            max_dl = ((d,l),(ig,i_T,i_F))

    
    if len(max_dl[1][1])==0 or len(max_dl[1][2])==0:
        if verbose:
            print("Split failed, couldn't find a d,l that separates values :(")
        return None, 0
    
    if verbose:
        print("Max IG is in couple d=",format_timedelta(max_dl[0][0]),", l=",max_dl[0][1],", IG=",max_dl[1])

    return max_dl



In [9]:
#tests

dataset = load_diabetes_dataset(False)
observation_window = timedelta(days=+5)
waiting_window = timedelta(days=+5)
prediction_window = timedelta(days=+5)

dataset_X,dataset_Y, dataset_ST,dataset_V, dataset_F = compute_datasets(dataset[:50],observation_window,waiting_window,prediction_window)
indexes = [3, 7, 11, 12, 13, 16, 22, 32, 33, 34, 35, 36, 40, 41, 44, 45, 46]
dataset_X = [dataset_X[i] for i in indexes]
dataset_Y = [dataset_Y[i] for i in indexes]
dataset_ST = [dataset_ST[i] for i in indexes]
dataset_V = [dataset_V[i] for i in indexes]

max_dl, max_ig = maximize_IG(dataset_X,dataset_Y,dataset_ST,range(len(dataset_X)),verbose=False,howmany=30,random_sampling=False)

print(f"{format_timedelta(max_dl[0]),max_dl[1],max_ig}")
print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V)

-- DS loader
	Skipped 46 items for formatting issues in data file. 70 loaded.
-- DS builder
	2 entries unsuitable for selected windows.
	Final dataset size: 48. Classes: 20|28, entropy 0.98
('23h 13m 9s', '65', (0.1861381990467907, [2, 3, 5, 7, 8, 9, 12, 16], [0, 1, 4, 6, 10, 11, 13, 14, 15]))
--DS state
index	X	Y	ST			V
0	l.41	0	1990-08-19 17:00:00	None
1	l.24	1	1990-07-31 12:09:00	None
2	l.46	1	1990-07-22 09:53:00	None
3	l.58	1	1990-09-04 05:53:00	None
4	l.38	1	1991-03-11 18:15:00	None
5	l.40	1	1990-07-13 09:48:00	None
6	l.37	0	1991-04-27 23:02:00	None
7	l.45	1	1990-08-23 07:19:00	None
8	l.52	1	1990-09-11 18:00:00	None
9	l.10	1	1991-03-28 15:35:00	None
10	l.47	1	1991-04-26 06:17:00	None
11	l.55	0	1991-06-11 18:05:00	None
12	l.52	1	1990-07-13 11:36:00	None
13	l.53	1	1990-08-26 17:26:00	None
14	l.52	1	1991-05-04 01:05:00	None
15	l.39	1	1991-07-05 20:47:00	None
16	l.36	1	1990-07-16 11:40:00	None
# entries: 17, entropy=0.672


## tree

In [51]:
from treelib import Node, Tree
class SequenceTree(Tree):
    def __init__(self, tree=None, deep=False, node_class=None, identifier=None):
        super(SequenceTree, self).__init__(tree=tree, deep=deep, node_class=node_class, identifier=identifier)

    def create_node(self, tag=None, identifier=None, parent=None, data=None,branch=None):
        """
        Create a child node for the given @parent node. If ``identifier`` is absent,
        a UUID will be generated automatically.
        """
        new_node = super(SequenceTree, self).create_node(tag=tag, identifier=identifier, parent=parent, data=data)
        
        siblings = super(SequenceTree,self).siblings(new_node.identifier)
        

        if len(super(SequenceTree,self).siblings(new_node.identifier))>=2:
           raise ValueError("Parent node already has maximum number of children")

        if branch in [x.data["branch"] for x in siblings]:
           raise ValueError(f"Parent node already has a {branch} branch")
        return new_node
    
    def display(self):
        print(self.show(stdout=False))
    
    def create_node_event(self,data,parent=None,branch=None):
        branch_f = "" if (branch is None) else branch+" "
        tag =  "\x1b[32m⬤\x1b[0m" +" "+ branch_f +"[" + str(data[1])+", " + format_timedelta(data[0])+ "] "  

        return     self.create_node(tag,data=(branch,{"classification":data}),parent=parent,branch=branch)


def create_node_event(tree,data,parent=None,branch=None,entropy="",size=0,ig="",index=""):
    branch_f = "" if (branch is None) else str(branch)+" "
    tag =  f"\x1b[32m⬤ {branch_f} ({str(data[1])},{format_timedelta(data[0])})\x1b[0m [e={entropy:4.2} ig={ig:4.2}] [n={size}] - {index}"
    data = {"branch":branch, "dl":(data[0],data[1]),"entropy":entropy,"ig":ig,"index":index}

    return     tree.create_node(tag,data=data,parent=parent,branch=branch)

def create_node_class(tree,classification,parent=None,branch=None,entropy="",size=0,index=""):
    branch_f = "" if (branch is None) else str(branch)+" "
    data = {"branch":branch, "class":classification,"entropy":entropy,"index":index}

    tag =  f"\x1b[33m◆ {branch_f} {classification} \x1b[0m- [e={float(entropy):2.2}] [n={size}] - {index}"

    return     tree.create_node(tag,data=data,parent=parent,branch=branch)

def create_node_value(tree,data,parent=None,branch=None,entropy="",size=0,ig="",index=""):
    branch_f = "" if (branch is None) else branch+" "
    tag =  f"\x1b[31m■ {branch_f} {data} \x1b[0m- [e={float(entropy):2.2}] [n={size}] - {index}"
 
    return     tree.create_node(tag,data=(branch,{"value":data}),parent=parent,branch=branch)


## let's start

In [53]:
tree = SequenceTree()

# prepare
dataset = load_diabetes_dataset(False)
observation_window = timedelta(days=+3)
waiting_window = timedelta(days=+0)
prediction_window = timedelta(days=+10)

dataset_X,dataset_Y, dataset_ST,dataset_V, dataset_F = compute_datasets(dataset[:],observation_window,waiting_window,prediction_window,)
#print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V)

def perform_event_test(max_dl,indexes,dataset_X,dataset_ST,dataset_V,verbose=False):
    # divide dataset in t and f...
    i_T = []
    i_F = []

    d,l = max_dl
    old_dataset_ST=dataset_ST
    
    for i in range(0,len(dataset_X)):
        entry = dataset_X[i]

        found=False

        for item in entry:
            #starting from the starting time, see if it exists an item with timestamp < d and label == l
            
            if(found is False and item[0]>= old_dataset_ST[i] and item[0]<=(old_dataset_ST[i]+d) and item[1]==l ): # If i'm over starting time
                found=True
                dataset_ST[i] = item[0]
                dataset_V[i] = item[2]

        if(found):
            i_T.append(i)
        else:
            i_F.append(i)
            
    return i_T,i_F,dataset_ST,dataset_V



def fit(dataset_X,dataset_Y,dataset_V,dataset_ST,max_depth,parent=None,branch=None,depth=0,indexes="",print_steps=False):


    if print_steps and (len(tree.all_nodes()) != 0):
        print(tree)

    # If all the classes are the same, make a class node
    if (len(dataset_X)==1) or (all(element == dataset_Y[0] for element in dataset_Y)):
        if print_steps:
            print(f"{depth,branch} All classes are the same!")
        create_node_class(tree=tree,classification=dataset_Y[0],parent=parent,branch=branch,entropy=compute_entropy(dataset_Y),size=len(dataset_Y),index=indexes)
        #print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V)
        return

    #1. get best event.
    max_dl, max_ig = maximize_IG(dataset_X,dataset_Y,dataset_ST,range(len(dataset_X)),False,howmany=30,random_sampling=False)



    
    # If classes are different but no split value is found, make a class node with majority
    if max_dl is None:
        if print_steps:
            print(f"{depth,branch} Couldn't find split node!")

        # Split not possible, make a class node!
        y_1 = sum(1 for c in dataset_Y if c == 1)
        y_0 = sum(1 for c in dataset_Y if c == 0)
        estimated_class = 1 if y_1>y_0 else 0    
        create_node_class(tree,str(estimated_class)+"NO SPLIT FOUND",parent,branch,compute_entropy(dataset_Y),len(dataset_Y),indexes)

    elif depth > max_depth:
        if (print_steps):
            print("Reached maximum depth!")
        
        y_1 = sum(1 for c in dataset_Y if c == 1)
        y_0 = sum(1 for c in dataset_Y if c == 0)
        estimated_class = 1 if y_1>y_0 else 0  
        estimated_class =   str(estimated_class)+" MAX DEPTH REACHED"
        return create_node_class(tree,estimated_class,parent,branch,compute_entropy(dataset_Y),len(dataset_Y),indexes)
    
    else:

        node = create_node_event(tree,max_dl,parent,branch,compute_entropy(dataset_Y),len(dataset_Y),max_ig[0])
        #print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F)


        i_T, i_F, dataset_ST, dataset_V=perform_event_test(max_dl,indexes,dataset_X,dataset_ST,dataset_V)
        fit([dataset_X[i] for i in i_T],[dataset_Y[i] for i in i_T],[dataset_V[i] for i in i_T],[dataset_ST[i] for i in i_T],max_depth,node,"t",depth+1,[indexes[i] for i in i_T])
        fit([dataset_X[i] for i in i_F],[dataset_Y[i] for i in i_F],[dataset_V[i] for i in i_F],[dataset_ST[i] for i in i_F],max_depth,node,"f",depth+1,[indexes[i] for i in i_F])
        
#print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V)
fit(dataset_X,dataset_Y,dataset_V,dataset_ST,max_depth=5,indexes=range(0,len(dataset_X)),print_steps=True)



print("--Final sequence tree")
if len(tree.all_nodes()) != 0:
    print(tree)



-- DS loader
	Skipped 46 items for formatting issues in data file. 70 loaded.
-- DS builder
	2 entries unsuitable for selected windows.
	Final dataset size: 68. Classes: 31|37, entropy 0.994
--Final sequence tree
⬤  (48,2d 17h 1m 1s) [e=0.99 ig=0.21] [n=68] - 
├── ⬤ f  (62,6h 57m 58s) [e=0.95 ig= 0.1] [n=46] - 
│   ├── ⬤ f  (34,2d 17h 1m 1s) [e=0.81 ig=0.17] [n=32] - 
│   │   ├── ⬤ f  (65,1d 15h 26m 17s) [e=0.99 ig= 0.4] [n=11] - 
│   │   │   ├── ⬤ f  (57,23h 3m 52s) [e=0.81 ig=0.31] [n=8] - 
│   │   │   │   ├── ⬤ f  (33,2h 17m 48s) [e= 1.0 ig=0.31] [n=4] - 
│   │   │   │   │   ├── ◆ f  1 - [e=0.0] [n=1] - [24]
│   │   │   │   │   └── ◆ t  0 MAX DEPTH REACHED - [e=0.92] [n=3] - [2, 56, 60]
│   │   │   │   └── ◆ t  0 - [e=0.0] [n=4] - [21, 48, 59, 62]
│   │   │   └── ◆ t  1 - [e=0.0] [n=3] - [35, 61, 64]
│   │   └── ⬤ t  (58,2h 19m 19s) [e=0.45 ig=0.18] [n=21] - 
│   │       ├── ⬤ f  (57,11h 36m 36s) [e=0.29 ig=0.15] [n=20] - 
│   │       │   ├── ⬤ t  (33,2h 12m 58s) [e=0.92 ig=0.92] [n